## Librerías de Teoría de Juegos: Gambit para juegos no cooperativos

In [13]:
!pip3 install pygambit

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


### One-shot trust game

In [2]:
import pygambit as gbt

g = gbt.Game.new_tree(players=["Buyer", "Seller"],
                      title="One-shot trust game, after Kreps (1990)")

In [15]:
g.root

Node(game=Game(title='One-shot trust game, after Kreps (1990)'), path=[])

In [16]:
g.root.children

NodeChildren(parent=Node(game=Game(title='One-shot trust game, after Kreps (1990)'), path=[]))

In [17]:
g.append_move(g.root, "Buyer", ["Trust", "Not trust"])

g.append_move(g.root.children[0], "Seller", ["Honor", "Abuse"])

In [18]:
g.set_outcome(g.root.children[0].children[0], g.add_outcome([1, 1], label="Trustworthy"))
g.set_outcome(g.root.children[0].children[1], g.add_outcome([-1, 2], label="Untrustworthy"))
g.set_outcome(g.root.children[1], g.add_outcome([0, 0], label="Opt-out"))

In [19]:
result = gbt.nash.lcp_solve(g)
result

NashComputationResult(method='lcp', rational=True, use_strategic=False, equilibria=[[[[Rational(0, 1), Rational(1, 1)]], [[Rational(0, 1), Rational(1, 1)]]]], parameters={'stop_after': 0, 'max_depth': 0})

In [20]:
eqm = result.equilibria[0]

In [21]:
eqm['Buyer']

[[Rational(0, 1), Rational(1, 1)]]

In [22]:
eqm['Seller']

[[Rational(0, 1), Rational(1, 1)]]

### Póker de una carta (suma constante)

Hay una baraja de cartas, con igual número de cartas de Rey y Reina. 

El juego comienza cuando cada jugador pone $1.

Se reparte una carta al azar a Alice; Alice observa su carta pero Bob no. 

Después de que Alice observe su carta, puede elegir subir la apuesta o retirarse. Si decide retirarse, Bob gana el dinero y el juego termina.
Si decide subir, añade otro $1. 

Después, Bob elige igualar o retirarse. 

Al final, Alice revela su carta. Si tiene un rey, gana; si tiene una reina, entonces Bob gana.


In [4]:
g = gbt.Game.new_tree(players=["Alice", "Bob"],
                      title="One card poker game, after Myerson (1991)")

g.append_move(g.root, g.players.chance, ["King", "Queen"])

for node in g.root.children:
    g.append_move(node, "Alice", ["Raise", "Fold"])

g.append_move(g.root.children[0].children[0], "Bob", ["Meet", "Pass"])

g.append_infoset(g.root.children[1].children[0],
                 g.root.children[0].children[0].infoset)

alice_winsbig = g.add_outcome([2, -2], label="Alice wins big")
alice_wins = g.add_outcome([1, -1], label="Alice wins")
bob_winsbig = g.add_outcome([-2, 2], label="Bob wins big")
bob_wins = g.add_outcome([-1, 1], label="Bob wins")

g.set_outcome(g.root.children[0].children[0].children[0], alice_winsbig)
g.set_outcome(g.root.children[0].children[0].children[1], alice_wins)
g.set_outcome(g.root.children[0].children[1], bob_wins)
g.set_outcome(g.root.children[1].children[0].children[0], bob_winsbig)
g.set_outcome(g.root.children[1].children[0].children[1], alice_wins)
g.set_outcome(g.root.children[1].children[1], bob_wins)

Infoset: https://gambitproject.readthedocs.io/en/latest/api/pygambit.gambit.Infoset.html#pygambit.gambit.Infoset

### Encontrando equilibrios de Nash

<img src="./imgs/nash.png">

In [5]:
result = gbt.nash.lcp_solve(g)
result

NashComputationResult(method='lcp', rational=True, use_strategic=False, equilibria=[[[[Rational(1, 1), Rational(0, 1)], [Rational(1, 3), Rational(2, 3)]], [[Rational(2, 3), Rational(1, 3)]]]], parameters={'stop_after': 0, 'max_depth': 0})

In [6]:
eqm = result.equilibria[0]

In [7]:
eqm['Alice'] 

# Resultados:
# --- Primer InfoSet ----
# Si tiene Rey, siempre apuesta
# --- Segundo InfoSet ----
# Si no tiene Rey, bluffea un tercio de las veces

[[Rational(1, 1), Rational(0, 1)], [Rational(1, 3), Rational(2, 3)]]

In [28]:
eqm['Bob']

# Resultados:
# Bob iguala dos tercios de las veces


[[Rational(2, 3), Rational(1, 3)]]

In [8]:
eqm['Bob']['Meet']

Rational(2, 3)

Como se trata de un equilibrio, el hecho de que Bob aleatorice en su InfoSet debe significar que es indiferente entre las dos acciones.


Veamos los payoffs:

In [31]:
{action: eqm.action_value(action) for action in g.players["Bob"].infosets[0].actions}

{Action(infoset=Infoset(player=Player(game=Game(title='One card poker game, after Myerson (1991)'), label='Bob'), number=0), label='Meet'): Rational(-1, 1),
 Action(infoset=Infoset(player=Player(game=Game(title='One card poker game, after Myerson (1991)'), label='Bob'), number=0), label='Pass'): Rational(-1, 1)}

### Resolución general

**gnm_solve()** se puede aplicar a cualquier juego con cualquier número de jugadores y utiliza un proceso de seguimiento de trayectorias en aritmética de punto flotante, por lo que devuelve perfiles con probabilidades expresadas como números de punto flotante.

In [32]:
result = gbt.nash.gnm_solve(g)
result

NashComputationResult(method='gnm', rational=False, use_strategic=True, equilibria=[[[0.33333333333866677, 0.6666666666613335, 0.0, 0.0], [0.6666666666559997, 0.3333333333440004]]], parameters={'perturbation': [[1.0, 0.0, 0.0, 0.0], [1.0, 0.0]], 'end_lambda': -10.0, 'steps': 100, 'local_newton_interval': 3, 'local_newton_maxits': 10})

Algunos métodos para calcular equilibrios de Nash operan utilizando aritmética de punto flotante y generan perfiles de equilibrio candidatos mediante aproximaciones sucesivas. 

Por lo tanto, los resultados de estos métodos son, en general, 𝜀-equilibrios, para algún ε positivo.